In [1]:
import arcgis as ag
import pandas as pd
import datetime

In [2]:
#credit conversions 
confac_features = 0.024 # units of dollar per MB * months
confac_data =  0.00012 # units of dollar per MB * months
confac_month_day = 730/24 # units of days / month - based on hourly calculation
confac_price_credit = 0.1 #units of dollar per credit

In [3]:
#create GIS instance - note you need to replace username and password
gis = ag.GIS("https://www.arcgis.com","username","password")

In [4]:
df = pd.DataFrame(data=None,columns=['item_ID','title','item_type','owner','access_level','fileSize_MB','hosting_daily_credits','hostingFee_daily','hostingFee_annualy','total_views','date_created','date_modified'])

In [5]:
#query our GIS content and assign to dataframe
items = gis.content.search(query="*",max_items=10000)
for item in items:
    if item.type == "Feature Service":
        hosting_credits = (item.size/1024/1024*confac_features/confac_month_day/confac_price_credit)
        hosting_daily = (item.size/1024/1024*confac_features/confac_month_day)
        hosting_annual = (item.size/1024/1024*confac_features*12)
    else:
        hosting_credits = (item.size/1024/1024*confac_data/confac_month_day/confac_price_credit)
        hosting_daily = (item.size/1024/1024*confac_data/confac_month_day)
        hosting_annual = (item.size/1024/1024*confac_data*12)
    df.loc[len(df)]=[
        item.id,
        item.title,
        item.type,
        item.owner,
        item.access,    
        item.size/1024/1024,
        hosting_credits,
        hosting_daily,
        hosting_annual,
        item.numViews,
        datetime.datetime.fromtimestamp(item.created/1000),
        datetime.datetime.fromtimestamp(item.modified/1000)
    ]

In [6]:
df.to_excel(R'.\AGOL_Report.xlsx',sheet_name="CreditReport")